This data feed is used in accordance with the usage terms of **buienradar.nl**, which allow free use provided proper attribution is given.  
Source: [buienradar.nl](https://www.buienradar.nl/)

In [33]:
import polars as pl
from src.skilltest.models import Measurement, WeatherStation
from src.skilltest.core.analysis import DataFrameConvertor

convertor = DataFrameConvertor()
measurements_df = convertor.get_dataframe(Measurement)
stations_df = convertor.get_dataframe(WeatherStation)
df = convertor.get_joined_dataframe(measurements_df, stations_df, on="stationid")

#### Question 5: Which weather station recorded the highest temperature?

In [ ]:
result = df.filter( #type: ignore
    pl.col("temperature") == df["temperature"].max()
).select([
    "stationname",
    "temperature",
    "timestamp"
])

print("Question 5 - Highest Temperature Station:")
print(result)

Question 5 - Highest Temperature Station:
shape: (1, 3)
┌────────────────────────┬─────────────┬────────────────────────────┐
│ stationname            ┆ temperature ┆ timestamp                  │
│ ---                    ┆ ---         ┆ ---                        │
│ str                    ┆ f64         ┆ str                        │
╞════════════════════════╪═════════════╪════════════════════════════╡
│ Meetstation Vlissingen ┆ 20.1        ┆ 2025-08-05 19:40:00.000000 │
└────────────────────────┴─────────────┴────────────────────────────┘


#### Question 6: What is the average temperature?

In [ ]:
avg_temp = df.select(pl.col("temperature").mean())[0, 0]

print(f"Question 6 - Average Temperature: {avg_temp:.2f}°C")

Question 6 - Average Temperature: 17.20°C


#### Question 7: What is the station with the biggest difference between feel temperature and the actual temperature?

In [32]:
df_with_diff = measurements_df.with_columns(
    (pl.col("feeltemperature") - pl.col("temperature")).abs().alias("temp_diff") #type: ignore
).filter(
    pl.col("temperature").is_not_null() &
    pl.col("feeltemperature").is_not_null()
)
avg_diff_by_station = df_with_diff.group_by("stationid").agg([
    pl.col("temp_diff").mean().alias("avg_temp_diff"),
    pl.col("temperature").mean().alias("avg_temperature"),
    pl.col("feeltemperature").mean().alias("avg_feeltemperature")
])

result_with_names = avg_diff_by_station.join(
    stations_df.select(["stationid", "stationname", "regio"]),
    on="stationid"
).sort("avg_temp_diff", descending=True)

result_with_names.head(1)

stationid,avg_temp_diff,avg_temperature,avg_feeltemperature,stationname,regio
i64,f64,f64,f64,str,str
6290,2.4,14.3,11.9,"""Meetstation Twente""","""Twente"""


#### Question 8: Which weather station is located in the North Sea?

In [31]:
result = df.filter( #type: ignore
    pl.col("regio").str.contains("(?i)noordzee|north sea") 
    | pl.col("stationname").str.contains( 
        "(?i)platform|zeeplatform|europlatform|k13"
    )
).select(["stationid", "stationname", "lat", "lon", "regio"])

print("Question 8 - North Sea Station(s):")
print(result)

Question 8 - North Sea Station(s):
shape: (1, 5)
┌───────────┬─────────────────────────────┬───────┬──────┬──────────┐
│ stationid ┆ stationname                 ┆ lat   ┆ lon  ┆ regio    │
│ ---       ┆ ---                         ┆ ---   ┆ ---  ┆ ---      │
│ i64       ┆ str                         ┆ f64   ┆ f64  ┆ str      │
╞═══════════╪═════════════════════════════╪═══════╪══════╪══════════╡
│ 6239      ┆ Meetstation Zeeplatform F-3 ┆ 54.85 ┆ 4.73 ┆ Noordzee │
└───────────┴─────────────────────────────┴───────┴──────┴──────────┘
